# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#imports csv file created in WeatherPy
cities = pd.read_csv("../Data/city_weather.csv", index_col = 0)
cities.head()

,City Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lumeje,-11.55,20.78,70.30,66.0,0.0,6.76,AO,1.601700e+09
1,Sawakin,19.11,37.33,86.18,62.0,0.0,4.54,SD,1.601700e+09
2,Rikitea,-23.12,-134.97,73.13,79.0,3.0,15.64,PF,1.601700e+09
3,Bethel,41.37,-73.41,46.99,93.0,1.0,4.70,US,1.601700e+09
4,Yellowknife,62.46,-114.35,41.00,93.0,40.0,5.82,CA,1.601700e+09


In [3]:
#finds max of Humidity for the humidity heatmap
cities["Humidity"].max()

100.0

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#sets the coordinates to plot on the map
coordinates = cities[["Lat", "Long"]]

#will be used for the heat map scale
humidity = cities["Humidity"]

In [5]:
#sets the figure templates
fig = gmaps.figure(layout={
        'width': '700px',
        'height': '600px',
        'padding': '1px',
        'border': '0px solid black',
        })

#creates the heat map layer
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 1.5, opacity = 0.4)

#adds the heat map layer to the map
fig.add_layer(heat_layer)

#saves the map - doesn't work
#plt.savefig("Map Images/Humidity Heatmap.png")

#displays the map
fig

Figure(layout=FigureLayout(border='0px solid black', height='600px', padding='1px', width='700px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#narrowing down the dataframe to fit the criteria of colder weather but not much wind because I don't like wind
hotel_df = cities[(cities["Max Temp"] < 40) & 
                       (cities["Max Temp"] >= 32) &
                        (cities["Humidity"] > 85) &
                        (cities["Wind Speed"] < 10) 
                        ]

In [7]:
#shows the counts of the above criteria to see how much it narrowed down by
hotel_df.count()

City Name     13
Lat           13
Long          13
Max Temp      13
Humidity      13
Cloudiness    13
Wind Speed    13
Country       13
Date          13
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#adds a new column for the hotel name
hotel_df["Hotel Name"] = ""

C:\Users\maco_\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
#sets the base url as the gmaps api
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#sets the parameters for the api call
params = {"radius": "5000",
         "type": "lodging",
         "key" : g_key,}

#prints a heading 
print("Finding hotel data")
print("--------------------------------")

#for loop with iterrows so it loops through the rows one at a time 
#also uses the index to set the row as a series so we can add to the row
for index, row in hotel_df.iterrows():
    
#sets the lat and long values for the hotel search    
    lat = row["Lat"]   
    long = row["Long"]
    name = row["City Name"]
    country = row["Country"]

#adds the lat and long to the parameters
    params["location"] = f"{lat},{long}"

#calls the API    
    response = requests.get(base_url, params=params).json()
     
#creates an exception if it cannot find the closest hotel. if it does adds the hotel name to the dataframe  
    try:
        print(f"Hotel found within 5000m of {name}, {country}.")
     
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]['name']
    
    except(KeyError, IndexError):    
         print(f"Hotel not found within 5000m of {name}, {country}...skipping")   
            
#prints a completion statement once the loop is finished            
print("--------------------------------")          
print("Data Retrieval Complete")  
print("--------------------------------")

Finding hotel data
--------------------------------
Hotel found within 5000m of Menomonie, US.


C:\Users\maco_\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel found within 5000m of Valparaiso, CL.
Hotel found within 5000m of La Ronge, CA.
Hotel found within 5000m of Geraldton, CA.
Hotel found within 5000m of Upernavik, GL.
Hotel found within 5000m of Aykhal, RU.
Hotel not found within 5000m of Aykhal, RU...skipping
Hotel found within 5000m of Longyearbyen, SJ.
Hotel found within 5000m of Husavik, IS.
Hotel found within 5000m of Flin Flon, CA.
Hotel found within 5000m of Shelburne, CA.
Hotel found within 5000m of Tasiilaq, GL.
Hotel found within 5000m of Paamiut, GL.
Hotel found within 5000m of Narsaq, GL.
--------------------------------
Data Retrieval Complete
--------------------------------


In [10]:
#replaces any blank rows with Null (nan)
hotel_df.replace("", np.nan, inplace = True)

#drops the rows that were unable to find the closest hotel
hotel_df_final = hotel_df.dropna()

C:\Users\maco_\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [11]:
hotel_df_final

,City Name,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
47,Menomonie,44.88,-91.92,33.01,88.0,1.0,4.16,US,1.601700e+09,Best Western Plus Menomonie Inn & Suites
58,Valparaiso,-33.04,-71.63,39.20,100.0,0.0,2.24,CL,1.601700e+09,Hotel Ultramar
87,La Ronge,55.10,-105.28,35.60,93.0,20.0,4.88,CA,1.601700e+09,Northland Motor Hotel
129,Geraldton,49.73,-86.95,32.00,96.0,90.0,1.45,CA,1.601700e+09,Victoria Bed & Breakfast
137,Upernavik,72.79,-56.15,32.85,86.0,98.0,3.98,GL,1.601700e+09,Gina's Guesthouse - Upernavik
173,Longyearbyen,78.22,15.64,32.00,92.0,75.0,4.70,SJ,1.601700e+09,"Radisson Blu Polar Hotel, Spitsbergen"
177,Husavik,66.04,-17.34,32.00,93.0,90.0,7.11,IS,1.601700e+09,Bjarnabúð
195,Flin Flon,54.77,-101.86,35.51,94.0,82.0,3.78,CA,1.601700e+09,Victoria Inn Flin Flon
209,Shelburne,44.08,-80.20,37.40,86.0,20.0,3.36,CA,1.601700e+09,The Retreat at the Farm
262,Tasiilaq,65.61,-37.64,33.80,86.0,100.0,4.70,GL,1.601700e+09,The Red House


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_final.iterrows()]
locations = hotel_df_final[["Lat", "Long"]]

In [13]:
#sets the figure templates
fig = gmaps.figure(layout={
        'width': '700px',
        'height': '600px',
        'padding': '1px',
        'border': '0px solid black'})

#creates the heat map layer
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 1.5, opacity = 0.4)

#Add marker layer ontop of heat ma
markers = gmaps.marker_layer(locations , info_box_content = hotel_info)

#adds the heat map layer to the map
fig.add_layer(heat_layer)
fig.add_layer(markers)

#saves the map - doesn't work properly
#plt.savefig("Map Images/Humidity Heatmap with Hotels.png")

#displays the map
fig

Figure(layout=FigureLayout(border='0px solid black', height='600px', padding='1px', width='700px'))